In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [4]:
dataset = tfds.load('imdb_reviews',as_supervised=True)

In [5]:
train_dataset,test_dataset=dataset['train'],dataset['test']
batch_size=32
train_dataset=train_dataset.shuffle(10000).batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

In [6]:
example,label=next(iter(train_dataset))
print('Text:\n',example.numpy()[0])
print('\nLabel: ', label.numpy()[0])

Text:
 b'THE RINGMASTER stars Jerry Springer as a TV talkshow host called Jerry , but it`s not THE JERRY SPRINGER SHOW , his guests are trailer trash , but not the trailer trash you get on THE JERRY SPRINGER SHOW, they attack one another , but not like on.....What is the point of making a movie about THE JERRY SPRINGER show and pretending it`s not THE JERRY SPRINGER SHOW ? And on top of that this is a very boring film'

Label:  0


text visualization 

In [7]:
vectorize_layer=tf.keras.layers.TextVectorization(output_mode='int',output_sequence_length=100)
vectorize_layer.adapt(train_dataset.map(lambda x, y: x))

In [8]:
vectorize_layer.adapt(train_dataset.map(lambda x,y:x))

defining the model (BiLSTM layers)

In [11]:
model=tf.keras.Sequential([vectorize_layer,tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()),
                                                                     64,mask_zero=True),
                                                                     
                                                                     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
                                                                     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                                                                     tf.keras.layers.Dense(64,activation='relu'),
                                                                     tf.keras.layers.Dense(1)])
model.build(input_shape=(None),)



In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history=model.fit(train_dataset,epochs=3,validation_data=test_dataset,)

Epoch 1/3
